<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #04



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #04

<br>
Facilities <br>
$|I| = 50$ 
<br>
<br>
Clients <br>
$|J| = 50$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [10]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcap133.txt'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))



# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2550 rows, 2550 columns and 7500 nonzeros
Model fingerprint: 0x8adc286f
Variable types: 2500 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3062847.1500
Presolve removed 2094 rows and 2069 columns
Presolve time: 0.03s
Presolved: 456 rows, 481 columns, 1258 nonzeros
Found heuristic solution: objective 1025990.0875
Variable types: 289 continuous, 192 integer (192 binary)

Root relaxation: objective 8.930767e+05, 159 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    893076.71250 893076.713  0.00%     -

In [11]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_04_Simplex.lp')


Build a warehouse at location 6.

Build a warehouse at location 23.

Build a warehouse at location 25.

Build a warehouse at location 27.

Build a warehouse at location 34.

Build a warehouse at location 45.

Build a warehouse at location 46.

Build a warehouse at location 49.

Client 1 receives 100.0 % of its demand  from Warehouse 25 .

Client 2 receives 100.0 % of its demand  from Warehouse 25 .

Client 3 receives 100.0 % of its demand  from Warehouse 6 .

Client 4 receives 100.0 % of its demand  from Warehouse 49 .

Client 5 receives 100.0 % of its demand  from Warehouse 25 .

Client 6 receives 100.0 % of its demand  from Warehouse 6 .

Client 7 receives 100.0 % of its demand  from Warehouse 34 .

Client 8 receives 100.0 % of its demand  from Warehouse 23 .

Client 9 receives 100.0 % of its demand  from Warehouse 25 .

Client 10 receives 100.0 % of its demand  from Warehouse 25 .

Client 11 receives 100.0 % of its demand  from Warehouse 23 .

Client 12 receives 100.0 % of its dema

<br>
<br>

<a id='random'></a>
### Random Heuristic 


In [12]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 1, 3, 4, 5, 6, 7, 10, 12, 13, 14, 15, 17, 18, 19, 22, 24, 25, 26, 27, 28, 29, 32, 34, 36, 37, 38, 39, 40, 43, 45, 48, 49]
2393392.0875000004


In [13]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 2.

Build a warehouse at location 4.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 11.

Build a warehouse at location 13.

Build a warehouse at location 14.

Build a warehouse at location 15.

Build a warehouse at location 16.

Build a warehouse at location 18.

Build a warehouse at location 19.

Build a warehouse at location 20.

Build a warehouse at location 23.

Build a warehouse at location 25.

Build a warehouse at location 26.

Build a warehouse at location 27.

Build a warehouse at location 28.

Build a warehouse at location 29.

Build a warehouse at location 30.

Build a warehouse at location 33.

Build a warehouse at location 35.

Build a warehouse at location 37.

Build a warehouse at location 38.

Build a warehouse at location 39.

Build a warehouse at location 40.

Build a warehouse at locat

In [14]:
len(facs)

33

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [15]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 0): 2609.75, (1, 1): 1148.4, (2, 2): 4914.0, (3, 3): 11548.3375, (4, 4): 566.525, (5, 5): 3514.7125, (6, 6): 28499.25, (7, 7): 6370.65, (8, 8): 697.95, (9, 9): 464.4, (10, 10): 12638.5, (11, 11): 1231.7, (12, 12): 5185.975, (13, 13): 1217.2875, (14, 14): 7310.8125, (15, 15): 8135.7, (16, 16): 1531.15, (17, 17): 11423.1, (18, 18): 2061.95, (19, 19): 3656.25, (20, 20): 390.45, (21, 21): 11166.8625, (22, 22): 0.0, (23, 23): 3553.0, (24, 24): 5270.65, (25, 25): 1634.45, (26, 26): 124051.2, (27, 27): 800.5875, (28, 28): 2958.275, (29, 29): 8049.9375, (30, 30): 2220.4875, (31, 31): 1199.45, (32, 32): 7346.625, (33, 33): 130895.4, (34, 34): 3176.875, (35, 35): 2992.05, (36, 36): 32993.1125, (37, 37): 10262.7875, (38, 38): 21387.9375, (39, 39): 2829.0, (40, 40): 5043.0, (41, 41): 8098.25, (42, 42): 5775.0, (43, 43): 7275.0, (44, 44): 68042.3625, (45, 45): 21302.8125, (46, 46): 5591.625, (47, 47): 1087.1875, (48, 48): 11949.9, (49, 49): 2000.775}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [16]:
#Get the greatest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 45): 11656.275, (1, 45): 6919.7625, (2, 38): 67393.2, (3, 44): 142841.7375, (4, 29): 2512.9375, (5, 38): 54202.0375, (6, 44): 266358.375, (7, 44): 108940.8375, (8, 29): 2869.7625, (9, 29): 2314.8, (10, 44): 477721.5625, (11, 44): 84557.9, (12, 44): 139911.375, (13, 38): 15131.1875, (14, 45): 54681.1875, (15, 29): 39867.75, (16, 44): 17709.925, (17, 44): 282712.3, (18, 44): 22071.0875, (19, 45): 16314.1875, (20, 44): 3108.875, (21, 45): 77038.2375, (22, 44): 50189.2125, (23, 38): 33421.0, (24, 44): 64804.575, (25, 38): 33190.2875, (26, 6): 406770.0, (27, 44): 53033.5125, (28, 38): 44699.475, (29, 38): 58502.8125, (30, 38): 23593.7625, (31, 44): 30199.575, (32, 45): 69570.3125, (33, 44): 1415639.4, (34, 45): 28075.9375, (35, 44): 27353.925, (36, 44): 394035.9625, (37, 44): 218284.7875, (38, 29): 73399.3125, (39, 44): 35444.5, (40, 44): 143032.0875, (41, 44): 93199.6875, (42, 29): 23103.4375, (43, 45): 47856.25, (44, 29): 224408.1375, (45, 29): 75966.2875, (46, 29): 21287.025, (47, 4

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [17]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 22): 5219.5, (1, 22): 2296.8, (2, 22): 9828.0, (3, 22): 23096.675, (4, 22): 1133.05, (5, 22): 7029.425, (6, 6): 45999.25, (7, 22): 12741.3, (8, 22): 1395.9, (9, 22): 928.8, (10, 22): 25277.0, (11, 22): 2463.4, (12, 22): 10371.95, (13, 22): 2434.575, (14, 22): 14621.625, (15, 22): 16271.4, (16, 22): 3062.3, (17, 22): 22846.2, (18, 22): 4123.9, (19, 22): 7312.5, (20, 22): 780.9, (21, 22): 22333.725, (22, 22): 0.0, (23, 22): 7106.0, (24, 22): 10541.3, (25, 22): 3268.9, (26, 26): 141551.2, (27, 22): 1601.175, (28, 22): 5916.55, (29, 22): 16099.875, (30, 22): 4440.975, (31, 22): 2398.9, (32, 22): 14693.25, (33, 33): 148395.4, (34, 22): 6353.75, (35, 22): 5984.1, (36, 36): 50493.1125, (37, 22): 20525.575, (38, 38): 38887.9375, (39, 22): 5658.0, (40, 22): 10086.0, (41, 22): 16196.5, (42, 22): 11550.0, (43, 22): 14550.0, (44, 44): 85542.3625, (45, 45): 38802.8125, (46, 22): 11183.25, (47, 22): 2174.375, (48, 22): 23899.8, (49, 22): 4001.55}
[33, 36, 6, 38, 44, 45, 22, 26]
122500
1065970.8

<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [18]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 45): 29156.275, (1, 45): 24419.7625, (2, 38): 84893.2, (3, 44): 160341.7375, (4, 29): 20012.9375, (5, 38): 71702.0375, (6, 44): 283858.375, (7, 44): 126440.8375, (8, 29): 20369.7625, (9, 29): 19814.8, (10, 44): 495221.5625, (11, 44): 102057.9, (12, 44): 157411.375, (13, 38): 32631.1875, (14, 45): 72181.1875, (15, 29): 57367.75, (16, 44): 35209.925, (17, 44): 300212.3, (18, 44): 39571.0875, (19, 45): 33814.1875, (20, 44): 20608.875, (21, 45): 94538.2375, (22, 44): 67689.2125, (23, 38): 50921.0, (24, 44): 82304.575, (25, 38): 50690.2875, (26, 6): 424270.0, (27, 44): 70533.5125, (28, 38): 62199.475, (29, 38): 76002.8125, (30, 38): 41093.7625, (31, 44): 47699.575, (32, 45): 87070.3125, (33, 44): 1433139.4, (34, 45): 45575.9375, (35, 44): 44853.925, (36, 44): 411535.9625, (37, 44): 235784.7875, (38, 29): 90899.3125, (39, 44): 52944.5, (40, 44): 160532.0875, (41, 44): 110699.6875, (42, 29): 40603.4375, (43, 45): 65356.25, (44, 29): 241908.1375, (45, 29): 93466.2875, (46, 29): 38787.025,